## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/michaelmanthey/Documents/Bootcamp/Module_6_Python_APIs/Challenge/Weather_Database/weather_data.csv", index_col=0)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Kaa-Khem,51.6833,94.7333,67.17,55,3,2.13,clear sky
1,Abu Samrah,35.3029,37.1841,77.02,58,0,6.38,clear sky
2,Faanui,-16.4833,-151.7500,78.96,66,6,14.14,clear sky
3,Lagoa,39.0500,-27.9833,72.90,82,7,17.60,clear sky
4,Busselton,-33.6500,115.3333,54.88,79,100,5.91,overcast clouds
...,...,...,...,...,...,...,...,...
692,Oparino,59.8518,48.2783,60.26,62,29,8.79,scattered clouds
693,Micheweni,-4.9667,39.8333,73.67,84,20,10.29,few clouds
694,Shizunai,42.3339,142.3669,66.16,65,100,11.03,overcast clouds
695,Karratha,-20.7377,116.8463,70.07,23,0,17.05,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would like on your vacation? "))
min_temp = float(input("What is the minimum temperature you would like on your vacation? "))

What is the maximum temperature you would like on your vacation? 90
What is the minimum temperature you would like on your vacation? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp)]
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,Abu Samrah,35.3029,37.1841,77.02,58,0,6.38,clear sky
2,Faanui,-16.4833,-151.7500,78.96,66,6,14.14,clear sky
13,Port Blair,11.6667,92.7500,77.04,88,75,13.80,broken clouds
15,Butaritari,3.0707,172.7902,81.75,77,30,7.83,light rain
16,Acapulco,16.8634,-99.8901,85.82,74,40,8.05,scattered clouds
...,...,...,...,...,...,...,...,...
685,Sabha,27.0377,14.4283,88.63,15,0,11.01,clear sky
686,Sibolga,1.7427,98.7792,78.13,84,82,1.92,broken clouds
687,Buala,-8.1450,159.5921,76.91,92,100,10.74,moderate rain
688,Bartica,6.4000,-58.6167,77.32,98,100,1.28,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
filtered_df.count()

City           227
Lat            227
Lng            227
Max Temp       227
Humidity       227
Cloudiness     227
Wind Speed     227
Description    227
dtype: int64

In [6]:
# 5. Create a new column "Hotel Name"
filtered_df["Hotel Name"] = ""
filtered_df

/Users/michaelmanthey/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
1,Abu Samrah,35.3029,37.1841,77.02,58,0,6.38,clear sky,
2,Faanui,-16.4833,-151.7500,78.96,66,6,14.14,clear sky,
13,Port Blair,11.6667,92.7500,77.04,88,75,13.80,broken clouds,
15,Butaritari,3.0707,172.7902,81.75,77,30,7.83,light rain,
16,Acapulco,16.8634,-99.8901,85.82,74,40,8.05,scattered clouds,
...,...,...,...,...,...,...,...,...,...
685,Sabha,27.0377,14.4283,88.63,15,0,11.01,clear sky,
686,Sibolga,1.7427,98.7792,78.13,84,82,1.92,broken clouds,
687,Buala,-8.1450,159.5921,76.91,92,100,10.74,moderate rain,
688,Bartica,6.4000,-58.6167,77.32,98,100,1.28,overcast clouds,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in filtered_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params = params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        filtered_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping.")
        
filtered_df
        

Hotel not found... Skipping.


/Users/michaelmanthey/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
1,Abu Samrah,35.3029,37.1841,77.02,58,0,6.38,clear sky,
2,Faanui,-16.4833,-151.7500,78.96,66,6,14.14,clear sky,Bora-Bora Pearl Beach Resort & Spa
13,Port Blair,11.6667,92.7500,77.04,88,75,13.80,broken clouds,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
15,Butaritari,3.0707,172.7902,81.75,77,30,7.83,light rain,Isles Sunset Lodge
16,Acapulco,16.8634,-99.8901,85.82,74,40,8.05,scattered clouds,HS Hotsson Smart Acapulco
...,...,...,...,...,...,...,...,...,...
685,Sabha,27.0377,14.4283,88.63,15,0,11.01,clear sky,Foundation Taaj Asala for Historical Research ...
686,Sibolga,1.7427,98.7792,78.13,84,82,1.92,broken clouds,Hotel Syariah CN Darussalam
687,Buala,-8.1450,159.5921,76.91,92,100,10.74,moderate rain,Maringe Lagoon Lodge
688,Bartica,6.4000,-58.6167,77.32,98,100,1.28,overcast clouds,Sloth Island Nature Resort


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = filtered_df[filtered_df["Hotel Name"] != ""]
        
clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
2,Faanui,-16.4833,-151.7500,78.96,66,6,14.14,clear sky,Bora-Bora Pearl Beach Resort & Spa
13,Port Blair,11.6667,92.7500,77.04,88,75,13.80,broken clouds,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
15,Butaritari,3.0707,172.7902,81.75,77,30,7.83,light rain,Isles Sunset Lodge
16,Acapulco,16.8634,-99.8901,85.82,74,40,8.05,scattered clouds,HS Hotsson Smart Acapulco
21,Jacareacanga,-6.2222,-57.7528,85.03,34,100,1.43,overcast clouds,Casa do Ian😎
...,...,...,...,...,...,...,...,...,...
685,Sabha,27.0377,14.4283,88.63,15,0,11.01,clear sky,Foundation Taaj Asala for Historical Research ...
686,Sibolga,1.7427,98.7792,78.13,84,82,1.92,broken clouds,Hotel Syariah CN Darussalam
687,Buala,-8.1450,159.5921,76.91,92,100,10.74,moderate rain,Maringe Lagoon Lodge
688,Bartica,6.4000,-58.6167,77.32,98,100,1.28,overcast clouds,Sloth Island Nature Resort


In [9]:
# 8a. Create the output File (CSV)
output_file = "/Users/michaelmanthey/Documents/Bootcamp/Module_6_Python_APIs/Challenge/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_df[["Lat", "Lng"]]
max_temp = clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))